#**Machine Learning - Desafío 7**

##*24 de octubre de 2021*

##**Francisca Pinto**

###**Ejercicio 1**

In [59]:
!pip install ipython-autotime
!pip install joblib

time: 5.77 s (started: 2021-10-24 20:40:17 +00:00)


In [60]:
#tiempo de ejecución
%load_ext autotime

#dataframe y arrays
import pandas as pd
import numpy as np

#estadística
import scipy.stats as stats

#gráficos

import matplotlib.pyplot as plt
import seaborn as sns

#gama matplotlib para facilitar escritura de código
%matplotlib inline


#separación entrenamiento validación
from sklearn.model_selection import train_test_split, GridSearchCV

#pipeline
from sklearn.pipeline import Pipeline

#clasificadores
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score
from sklearn.model_selection import cross_val_score

#preprocesamiento
from sklearn.preprocessing import StandardScaler, LabelEncoder, LabelBinarizer, PolynomialFeatures

#árboles de regresión
from sklearn.tree import DecisionTreeRegressor

#exportación de modelos
import joblib

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 23.6 ms (started: 2021-10-24 20:40:33 +00:00)


In [3]:
#AUX: montar archivos y carpetas desde Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 1.95 ms (started: 2021-10-24 16:48:53 +00:00)


In [5]:
#apertura de archivo.csv en Colab

path_colab = "/content/drive/MyDrive/ADL/3_Intro ML/Desafío 7 - Árboles de regresión/Apoyo Desafío - Árboles de regresión/ames_housing.csv"
df = pd.read_csv(path_colab)
df.drop(columns = "Unnamed: 0", inplace = True) #quitar columna automática y reemplazar df
df.head()

,MS_SubClass,MS_Zoning,Lot_Frontage,Lot_Area,Street,Alley,Lot_Shape,Land_Contour,Utilities,Lot_Config,Land_Slope,Neighborhood,Condition_1,Condition_2,Bldg_Type,House_Style,Overall_Qual,Overall_Cond,Year_Built,Year_Remod_Add,Roof_Style,Roof_Matl,Exterior_1st,Exterior_2nd,Mas_Vnr_Type,Mas_Vnr_Area,Exter_Qual,Exter_Cond,Foundation,Bsmt_Qual,Bsmt_Cond,Bsmt_Exposure,BsmtFin_Type_1,BsmtFin_SF_1,BsmtFin_Type_2,BsmtFin_SF_2,Bsmt_Unf_SF,Total_Bsmt_SF,Heating,Heating_QC,...,Electrical,First_Flr_SF,Second_Flr_SF,Low_Qual_Fin_SF,Gr_Liv_Area,Bsmt_Full_Bath,Bsmt_Half_Bath,Full_Bath,Half_Bath,Bedroom_AbvGr,Kitchen_AbvGr,Kitchen_Qual,TotRms_AbvGrd,Functional,Fireplaces,Fireplace_Qu,Garage_Type,Garage_Finish,Garage_Cars,Garage_Area,Garage_Qual,Garage_Cond,Paved_Drive,Wood_Deck_SF,Open_Porch_SF,Enclosed_Porch,Three_season_porch,Screen_Porch,Pool_Area,Pool_QC,Fence,Misc_Feature,Misc_Val,Mo_Sold,Year_Sold,Sale_Type,Sale_Condition,Sale_Price,Longitude,Latitude
0,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,141,31770,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,Gtl,North_Ames,Norm,Norm,OneFam,One_Story,Above_Average,Average,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112,Typical,Typical,CBlock,Typical,Good,Gd,BLQ,2,Unf,0,441,1080,GasA,Fair,...,SBrkr,1656,0,0,1656,1,0,1,0,3,1,Typical,7,Typ,2,Good,Attchd,Fin,2,528,Typical,Typical,Partial_Pavement,210,62,0,0,0,0,No_Pool,No_Fence,None,0,5,2010,WD,Normal,215000,-93.619754,42.054035
1,One_Story_1946_and_Newer_All_Styles,Residential_High_Density,80,11622,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,Gtl,North_Ames,Feedr,Norm,OneFam,One_Story,Average,Above_Average,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0,Typical,Typical,CBlock,Typical,Typical,No,Rec,6,LwQ,144,270,882,GasA,Typical,...,SBrkr,896,0,0,896,0,0,1,0,2,1,Typical,5,Typ,0,No_Fireplace,Attchd,Unf,1,730,Typical,Typical,Paved,140,0,0,0,120,0,No_Pool,Minimum_Privacy,None,0,6,2010,WD,Normal,105000,-93.619756,42.053014
2,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,81,14267,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,Gtl,North_Ames,Norm,Norm,OneFam,One_Story,Above_Average,Above_Average,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108,Typical,Typical,CBlock,Typical,Typical,No,ALQ,1,Unf,0,406,1329,GasA,Typical,...,SBrkr,1329,0,0,1329,0,0,1,1,3,1,Good,6,Typ,0,No_Fireplace,Attchd,Unf,1,312,Typical,Typical,Paved,393,36,0,0,0,0,No_Pool,No_Fence,Gar2,12500,6,2010,WD,Normal,172000,-93.619387,42.052659
3,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,93,11160,Pave,No_Alley_Access,Regular,Lvl,AllPub,Corner,Gtl,North_Ames,Norm,Norm,OneFam,One_Story,Good,Average,1968,1968,Hip,CompShg,BrkFace,BrkFace,None,0,Good,Typical,CBlock,Typical,Typical,No,ALQ,1,Unf,0,1045,2110,GasA,Excellent,...,SBrkr,2110,0,0,2110,1,0,2,1,3,1,Excellent,8,Typ,2,Typical,Attchd,Fin,2,522,Typical,Typical,Paved,0,0,0,0,0,0,No_Pool,No_Fence,None,0,4,2010,WD,Normal,244000,-93.617320,42.051245
4,Two_Story_1946_and_Newer,Residential_Low_Density,74,13830,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,OneFam,Two_Story,Average,Average,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0,Typical,Typical,PConc,Good,Typical,No,GLQ,3,Unf,0,137,928,GasA,Good,...,SBrkr,928,701,0,1629,0,0,2,1,3,1,Typical,6,Typ,1,Typical,Attchd,Fin,2,482,Typical,Typical,Paved,212,34,0,0,0,0,No_Pool,Minimum_Privacy,None,0,3,2010,WD,Normal,189900,-93.638933,42.060899


time: 1.83 s (started: 2021-10-24 16:48:53 +00:00)


###**Ejercicio 2**

Comentarios:

* Para verificar el tipo de dato asignado a cada atributo, primero se aplica el método <code>.info()</code>. Posteriormente, se iterará sobre cada columna del DataFrame para obtener: la cantidad de columnas numéricas, y las columnas que tienen variables categóricas (<code>dtype = object</code>) para corroborar con la muestra del DataFrame obtenida en <code>.info()</code>.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 81 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   MS_SubClass         2930 non-null   object 
 1   MS_Zoning           2930 non-null   object 
 2   Lot_Frontage        2930 non-null   int64  
 3   Lot_Area            2930 non-null   int64  
 4   Street              2930 non-null   object 
 5   Alley               2930 non-null   object 
 6   Lot_Shape           2930 non-null   object 
 7   Land_Contour        2930 non-null   object 
 8   Utilities           2930 non-null   object 
 9   Lot_Config          2930 non-null   object 
 10  Land_Slope          2930 non-null   object 
 11  Neighborhood        2930 non-null   object 
 12  Condition_1         2930 non-null   object 
 13  Condition_2         2930 non-null   object 
 14  Bldg_Type           2930 non-null   object 
 15  House_Style         2930 non-null   object 
 16  Overal

In [7]:
colname_numeric_elements = []
for colname, serie in df.iteritems():
  if pd.api.types.is_numeric_dtype(serie) == True:
    colname_numeric_elements.append(colname)

print(len(colname_numeric_elements))

35
time: 6.78 ms (started: 2021-10-24 16:48:55 +00:00)


In [8]:
colname_object_elements = []
for colname, serie in df.iteritems():
  if pd.api.types.is_numeric_dtype(serie) != True:
    colname_object_elements.append(colname)

print(colname_object_elements)

['MS_SubClass', 'MS_Zoning', 'Street', 'Alley', 'Lot_Shape', 'Land_Contour', 'Utilities', 'Lot_Config', 'Land_Slope', 'Neighborhood', 'Condition_1', 'Condition_2', 'Bldg_Type', 'House_Style', 'Overall_Qual', 'Overall_Cond', 'Roof_Style', 'Roof_Matl', 'Exterior_1st', 'Exterior_2nd', 'Mas_Vnr_Type', 'Exter_Qual', 'Exter_Cond', 'Foundation', 'Bsmt_Qual', 'Bsmt_Cond', 'Bsmt_Exposure', 'BsmtFin_Type_1', 'BsmtFin_Type_2', 'Heating', 'Heating_QC', 'Central_Air', 'Electrical', 'Kitchen_Qual', 'Functional', 'Fireplace_Qu', 'Garage_Type', 'Garage_Finish', 'Garage_Qual', 'Garage_Cond', 'Paved_Drive', 'Pool_QC', 'Fence', 'Misc_Feature', 'Sale_Type', 'Sale_Condition']
time: 3.49 ms (started: 2021-10-24 16:48:55 +00:00)


Comentarios:

* La cantidad de columnas de tipo numérico (<code>dtype = int64, float64</code>) coincide con lo observado en <code>.info()</code>.
* Ahora se utilizará la información extraída en el segundo loop (nombres de variables categóricas) para pasar tales columnas a <code>dummy</code> con <code>.get_dummies()</code>. Este método ya no requiere concatenar posteriormente las nuevas columnas, ni eliminar las que han sido transformadas, ya que lo realiza automáticamente.
* El DataFrame <code>dummy</code> tendrá un nuevo nombre para no confundirlo con el anterior.
* Para verificar la aplicación de <code>.get_dummies()</code> se aplicará nuevamente <code>.info()</code>.

In [9]:
df_dummy = pd.get_dummies(data = df,
                          columns = colname_object_elements,
                          drop_first = True,
                          dtype = "int")

time: 77.5 ms (started: 2021-10-24 16:48:55 +00:00)


In [10]:
df_dummy #se revisa para observar la correcta aplicación del método

,Lot_Frontage,Lot_Area,Year_Built,Year_Remod_Add,Mas_Vnr_Area,BsmtFin_SF_1,BsmtFin_SF_2,Bsmt_Unf_SF,Total_Bsmt_SF,First_Flr_SF,Second_Flr_SF,Low_Qual_Fin_SF,Gr_Liv_Area,Bsmt_Full_Bath,Bsmt_Half_Bath,Full_Bath,Half_Bath,Bedroom_AbvGr,Kitchen_AbvGr,TotRms_AbvGrd,Fireplaces,Garage_Cars,Garage_Area,Wood_Deck_SF,Open_Porch_SF,Enclosed_Porch,Three_season_porch,Screen_Porch,Pool_Area,Misc_Val,Mo_Sold,Year_Sold,Sale_Price,Longitude,Latitude,MS_SubClass_One_Story_1945_and_Older,MS_SubClass_One_Story_1946_and_Newer_All_Styles,MS_SubClass_One_Story_PUD_1946_and_Newer,MS_SubClass_One_Story_with_Finished_Attic_All_Ages,MS_SubClass_One_and_Half_Story_Finished_All_Ages,...,Garage_Finish_Unf,Garage_Qual_Fair,Garage_Qual_Good,Garage_Qual_No_Garage,Garage_Qual_Poor,Garage_Qual_Typical,Garage_Cond_Fair,Garage_Cond_Good,Garage_Cond_No_Garage,Garage_Cond_Poor,Garage_Cond_Typical,Paved_Drive_Partial_Pavement,Paved_Drive_Paved,Pool_QC_Fair,Pool_QC_Good,Pool_QC_No_Pool,Pool_QC_Typical,Fence_Good_Wood,Fence_Minimum_Privacy,Fence_Minimum_Wood_Wire,Fence_No_Fence,Misc_Feature_Gar2,Misc_Feature_None,Misc_Feature_Othr,Misc_Feature_Shed,Misc_Feature_TenC,Sale_Type_CWD,Sale_Type_Con,Sale_Type_ConLD,Sale_Type_ConLI,Sale_Type_ConLw,Sale_Type_New,Sale_Type_Oth,Sale_Type_VWD,Sale_Type_WD,Sale_Condition_AdjLand,Sale_Condition_Alloca,Sale_Condition_Family,Sale_Condition_Normal,Sale_Condition_Partial
0,141,31770,1960,1960,112,2,0,441,1080,1656,0,0,1656,1,0,1,0,3,1,7,2,2,528,210,62,0,0,0,0,0,5,2010,215000,-93.619754,42.054035,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1,80,11622,1961,1961,0,6,144,270,882,896,0,0,896,0,0,1,0,2,1,5,0,1,730,140,0,0,0,120,0,0,6,2010,105000,-93.619756,42.053014,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
2,81,14267,1958,1958,108,1,0,406,1329,1329,0,0,1329,0,0,1,1,3,1,6,0,1,312,393,36,0,0,0,0,12500,6,2010,172000,-93.619387,42.052659,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
3,93,11160,1968,1968,0,1,0,1045,2110,2110,0,0,2110,1,0,2,1,3,1,8,2,2,522,0,0,0,0,0,0,0,4,2010,244000,-93.617320,42.051245,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,74,13830,1997,1998,0,3,0,137,928,928,701,0,1629,0,0,2,1,3,1,6,1,2,482,212,34,0,0,0,0,0,3,2010,189900,-93.638933,42.060899,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,37,7937,1984,1984,0,3,0,184,1003,1003,0,0,1003,1,0,1,0,3,1,6,0,2,588,120,0,0,0,0,0,0,3,2006,142500,-93.604776,41.988964,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
2926,0,8885,1983,1983,0,2,324,239,864,902,0,0,902,1,0,1,0,2,1,5,0,2,484,164,0,0,0,0,0,0,6,2006,131000,-93.602680,41.988314,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
2927,62,10441,1992,1992,0,3,0,575,912,970,0,0,970,0,1,1,0,3,1,6,0,0,0,80,32,0,0,0,0,700,7,2006,132000,-93.606847,41.986510,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
2928,77,10010,1974,1975,0,1,123,195,1389,1389,0,0,1389,1,0,1,0,2,1,6,1,2,418,240,38,0,0,0,0,0,4,2006,170000,-93.600190,41.990921,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0


time: 135 ms (started: 2021-10-24 16:48:55 +00:00)


In [11]:
df_dummy.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 307 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Lot_Frontage                                           2930 non-null   int64  
 1   Lot_Area                                               2930 non-null   int64  
 2   Year_Built                                             2930 non-null   int64  
 3   Year_Remod_Add                                         2930 non-null   int64  
 4   Mas_Vnr_Area                                           2930 non-null   int64  
 5   BsmtFin_SF_1                                           2930 non-null   int64  
 6   BsmtFin_SF_2                                           2930 non-null   int64  
 7   Bsmt_Unf_SF                                            2930 non-null   int64  
 8   Total_Bsmt_SF                                  

Comentarios:

* Dada la corroboración de resultados, se puede continuar con el <code>Ejercicio 3</code>.

###**Ejercicio 3**

Comentarios:

* Se crean los conjuntos <code>train</code> y <code>test</code> que serán utilizados, con una semilla pseudoaleatoria para asegurar replicabilidad de los resultados.
* Se creará el modelo con <code>DecisionTreeRegressor</code> sin modificar los hiperparámetros.
* Posteriormente se genera el vector con las predicciones, y se crean las métricas respectivas para luego imprimirlas.

In [12]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(df_dummy.drop(columns = "Sale_Price"),
                                                    df_dummy["Sale_Price"],
                                                    test_size = 0.33,
                                                    random_state = 3748)

time: 20.1 ms (started: 2021-10-24 16:48:55 +00:00)


In [13]:
model1 = DecisionTreeRegressor().fit(X_train1, y_train1)
yhat1 = model1.predict(X_test1)

time: 93.8 ms (started: 2021-10-24 16:48:55 +00:00)


In [14]:
model1_mse = mean_squared_error(y_test1, yhat1)
model1_mae = median_absolute_error(y_test1, yhat1)
model1_r2_score = r2_score(y_test1, yhat1)

time: 6.73 ms (started: 2021-10-24 16:48:55 +00:00)


In [15]:
print(f"Modelo 1, Error Cuadrático Medio: {model1_mse.round(4)}", f"\nModelo 1, Mediana del Error Absoluto: {model1_mae.round(4)}", f"\nModelo 1, Puntaje R2: {model1_r2_score.round(4)}")

Modelo 1, Error Cuadrático Medio: 1721874584.4281 
Modelo 1, Mediana del Error Absoluto: 17300.0 
Modelo 1, Puntaje R2: 0.7425
time: 3.62 ms (started: 2021-10-24 16:48:55 +00:00)


In [16]:
np.sqrt(model1_mse)

41495.47667430907

time: 4.59 ms (started: 2021-10-24 16:48:55 +00:00)


Comentarios:

* Se corrobora dimensión de <code>MSE</code> extrayendo la raíz (ya que es un error cuadrático). Es un valor alto pero tiene sentido considerando el rango de valores de <code>Sale_Price</code>.
* <code>MAE</code> también tiene valores que corresponden al rango de valores del vector objetivo.
* <code>r2_score</code> muestra que el modelo explica un <code>77%</code> de la variabilidad del vector objetivo. Estos valores posteriormente se compararán con los de otros modelos ajustados.

###**Ejercicio 4**

Comentarios:

* Se revisará la importancia de los atributos con el método <code>feature_importances_</code>, aplicado al modelo ya realizado.
* Esta información se guardará en un solo DataFrame, que posteriormente será separado: en el primero, se conservarán solo los primeros 10 valores, y en el segundo todos los demás, esta última información será necesaria para posteriormente crear el nuevo DataFrame para que estas columnas puedan utilizarse al aplicar <code>.drop()</code>.

In [17]:
feature_importance_dict = {"Feature" : X_train1.columns,
                           "Feature_importance" : model1.feature_importances_}              

time: 2.03 ms (started: 2021-10-24 16:48:55 +00:00)


In [18]:
feature_importance_df = pd.DataFrame(data = feature_importance_dict)

time: 3.04 ms (started: 2021-10-24 16:48:55 +00:00)


In [19]:
feature_importance_df_10, feature_importance_df_11 = feature_importance_df.sort_values("Feature_importance",
                                  ascending = False)[:10], feature_importance_df.sort_values("Feature_importance",
                                  ascending = False)[10:]

time: 4.97 ms (started: 2021-10-24 16:48:55 +00:00)


In [58]:
feature_importance_df_10 #se imprime para corroborar

,Feature,Feature_importance
21,Garage_Cars,0.391990
2,Year_Built,0.147263
12,Gr_Liv_Area,0.137268
8,Total_Bsmt_SF,0.098647
9,First_Flr_SF,0.031704
1,Lot_Area,0.022153
33,Latitude,0.016814
32,Longitude,0.011962
3,Year_Remod_Add,0.009585
22,Garage_Area,0.009422


time: 17.5 ms (started: 2021-10-24 20:30:12 +00:00)


In [21]:
df_dummy_2 = df_dummy.drop(columns = feature_importance_df_11.iloc[:, 0])

time: 4.24 ms (started: 2021-10-24 16:48:55 +00:00)


In [22]:
df_dummy_2

,Lot_Area,Year_Built,Year_Remod_Add,Total_Bsmt_SF,First_Flr_SF,Gr_Liv_Area,Garage_Cars,Garage_Area,Sale_Price,Longitude,Latitude
0,31770,1960,1960,1080,1656,1656,2,528,215000,-93.619754,42.054035
1,11622,1961,1961,882,896,896,1,730,105000,-93.619756,42.053014
2,14267,1958,1958,1329,1329,1329,1,312,172000,-93.619387,42.052659
3,11160,1968,1968,2110,2110,2110,2,522,244000,-93.617320,42.051245
4,13830,1997,1998,928,928,1629,2,482,189900,-93.638933,42.060899
...,...,...,...,...,...,...,...,...,...,...,...
2925,7937,1984,1984,1003,1003,1003,2,588,142500,-93.604776,41.988964
2926,8885,1983,1983,864,902,902,2,484,131000,-93.602680,41.988314
2927,10441,1992,1992,912,970,970,0,0,132000,-93.606847,41.986510
2928,10010,1974,1975,1389,1389,1389,2,418,170000,-93.600190,41.990921


time: 41.5 ms (started: 2021-10-24 16:48:55 +00:00)


###**Ejercicio 5**

Comentarios:

* Se volverán a crear los conjuntos <code>train</code> y <code>test</code> sobre el nuevo DataFrame, que ahora tiene solo las columnas que más relevancia tienen en la capacidad predictiva del modelo.
* Posteriormente, se aplica el método <code>GridSearchCV</code> para buscar la mejor combinación de hiperparámetros para el segundo Árbol de Regresión. Los hiperparámetros que se busca evaluar son la cantidad máxima de atributos a incorporar en el árbol y su profundidad máxima.

In [23]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(df_dummy_2.drop(columns = "Sale_Price"),
                                                        df_dummy_2["Sale_Price"],
                                                        test_size = 0.33,
                                                        random_state = 3748)

time: 8.64 ms (started: 2021-10-24 16:48:55 +00:00)


In [24]:
model2_gridsearchcv = GridSearchCV(DecisionTreeRegressor(),
                                   {"max_features" : list(range(1, X_train2.shape[1] + 1)),
                                    "max_depth" : np.linspace(1, 32, 32)},
                                   cv = 5,
                                   n_jobs = -1).fit(X_train2, y_train2)

time: 14.6 s (started: 2021-10-24 16:48:55 +00:00)


In [25]:
model2_gridsearchcv.best_params_

{'max_depth': 5.0, 'max_features': 9}

time: 4.28 ms (started: 2021-10-24 16:49:10 +00:00)


In [26]:
model2_gridsearchcv.best_score_

0.7637374538775955

time: 4.04 ms (started: 2021-10-24 16:49:10 +00:00)


Comentarios:

* El rendimiento del modelo 2 es prácticamente el mismo que el del modelo 1 (variación va de 0.7425 a 0.7637, un aumento de un <code>2,9%</code>).
* Los hiperparámetros del mejor modelo, luego de aplicar <code>GridSearchCV</code> son:
1. <code>max_depth = 5</code>
2. <code>max_features = 9</code>
* Se guardará el mejor modelo con la librería <code>joblib</code>, siguiendo la información encontrada en [1].
* Para ello, se realizará todo el pre-procesamiento anterior de forma resumida en la celda siguiente, solo rescatando los nombres de las columnas y los hiperparámetros del trabajo previo, y posteriormente se generará el modelo y se guardará con el método <code>.dump()</code>.

In [57]:
url = "/content/drive/MyDrive/ADL/3_Intro ML/Desafío 7 - Árboles de regresión/Apoyo Desafío - Árboles de regresión/ames_housing.csv"
columns_names = ["Lot_Area", "Year_Built", "Year_Remod_Add", "Total_Bsmt_SF", "First_Flr_SF",
                 "Gr_Liv_Area", "Garage_Cars", "Garage_Area", "Sale_Price", "Longitude",
                 "Latitude"]
dataframe1 = pd.read_csv(url)
dataframe2 = dataframe1.drop(columns = "Unnamed: 0", inplace = False)
dataframe3 = pd.get_dummies(data = dataframe2,
                            drop_first = True,
                            dtype = "int")
dataframe4 = dataframe3.loc[ : , columns_names]
X_train_jl, X_test_jl, y_train_jl, y_test_jl = train_test_split(dataframe4.drop(columns = "Sale_Price"),
                                                                dataframe4["Sale_Price"],
                                                                test_size = 0.33,
                                                                random_state = 3748)
model_jl = DecisionTreeRegressor(max_depth = 5,
                                 max_features = 9).fit(X_train_jl, y_train_jl)
filename = "/content/drive/MyDrive/ADL/3_Intro ML/Desafío 7 - Árboles de regresión/Apoyo Desafío - Árboles de regresión/model_dectree_FP.sav"
joblib.dump(model_jl, filename)

['/content/drive/MyDrive/ADL/3_Intro ML/Desafío 7 - Árboles de regresión/Apoyo Desafío - Árboles de regresión/model_dectree_FP.sav']

time: 147 ms (started: 2021-10-24 18:42:35 +00:00)


Fuentes:

[1] <code>https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/</code>